In [1]:
import sqlite3
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from measure_disparity import Measure_disparity
from mitigate_disparity import relocate_icd9_section
from helpers import *
import lightgbm as lgb
import shap
import scipy
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
import warnings
warnings.filterwarnings("ignore")

seed = 0
random.seed(seed)
np.random.seed(seed)

db_path = '/media/jifangao/backup/mimic_iii_dataset/mimiciii.sqlite3'

def mean_confidence_interval(data, confidence=0.90):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m,3), round(m-h,3), round(m+h,3)

/home/jifangao/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# cases
con = sqlite3.connect(db_path)
query = """
        WITH t AS
        (
            SELECT *
            FROM (
                SELECT subject_id, hadm_id, dischtime, 
                JULIANDAY(MAX(dischtime)) AS last_dischtime, hospital_expire_flag
                FROM admissions
                GROUP BY subject_id
            )
            WHERE hospital_expire_flag = 0
        )
        SELECT t.*, p.dod
        FROM t
        LEFT JOIN patients p
        ON t.subject_id = p.subject_id
        WHERE JULIANDAY(p.dod) - last_dischtime > 1
        AND JULIANDAY(p.dod) - last_dischtime <= 30
        """

df_cases = pd.read_sql_query(query, con)
con.commit()
con.close()

# cohort
con = sqlite3.connect(db_path)
query = """
        SELECT *
        FROM (
            SELECT subject_id, hadm_id, dischtime, 
            JULIANDAY(MAX(dischtime)) AS last_dischtime, hospital_expire_flag
            FROM admissions
            GROUP BY subject_id
        )
        WHERE hospital_expire_flag = 0
        """

df_cohort = pd.read_sql_query(query, con)
con.commit()
con.close()

# df to save admission infos
con = sqlite3.connect(db_path)
query = """
        select ad.hadm_id, pt.subject_id,
        ad.ethnicity,
        (JulianDay(ad.admittime)-JulianDay(pt.dob))/365 as age,
        case when JulianDay(ad.dischtime)-JulianDay(ad.admittime)>30 then 1 else 0 end as long_adm,
        0 as re_adm,
        hospital_expire_flag as death
        from admissions ad, patients pt
        where pt.subject_id = ad.subject_id
        """

df_adm = pd.read_sql_query(query, con)
con.commit()
con.close()

# df to save admission infos
con = sqlite3.connect(db_path)
query = """
        select ad.hadm_id, pt.subject_id,
        ad.ethnicity,
        (JulianDay(ad.admittime)-JulianDay(pt.dob))/365 as age,
        case when JulianDay(ad.dischtime)-JulianDay(ad.admittime)>30 then 1 else 0 end as long_adm,
        0 as re_adm,
        hospital_expire_flag as death
        from admissions ad, patients pt
        where pt.subject_id = ad.subject_id
        """

df_adm = pd.read_sql_query(query, con)
con.commit()
con.close()

# dictionary to save patients and admission dates
dic_pat_adm = {}
con = sqlite3.connect(db_path)
query = """
        select subject_id, hadm_id, JulianDay(admittime), JulianDay(dischtime)
        from admissions
        """
c = con.cursor()
c.execute(query)
for row in c:
    if row[0] in df_cohort.subject_id.values:
        if row[0] in dic_pat_adm:
            dic_pat_adm[row[0]].append((row[1], row[2], row[3]))
        else:
            dic_pat_adm[row[0]] = [(row[1], row[2], row[3])]
con.commit()
con.close()

for i in dic_pat_adm:
    if len(dic_pat_adm[i]) == 1: continue
    dic_pat_adm[i].sort(key=lambda x: x[1])
    
# dictionary to save admisssions and daignosis
dic_admi_dgx = {}
con = sqlite3.connect(db_path)
query = """
        select *
        from diagnosis_icd
        """
c = con.cursor()
c.execute(query)
for row in c:
    if row[2] in dic_admi_dgx:
        dic_admi_dgx[row[2]].append(row[-1])#[:3])
    else:
        dic_admi_dgx[row[2]] = [row[-1]]
con.commit()
con.close()

# encode race groups
dic_race_encode = race_encoder_mimiciii()
df_adm["ETHNICITY_encoded"] = df_adm["ETHNICITY"].map(dic_race_encode)
df_adm["ETHNICITY_encoded"].value_counts()           
dic_pat_race = {}
for _, row in df_adm.iterrows():
    dic_pat_race[row['SUBJECT_ID']] = row['ETHNICITY_encoded']
    
# collect dignosis of each patient
pat_dgx = {}
loc = 150 # number of icd-9 sections
for i in dic_pat_adm:
    pat_dgx[i] = []
    count = 0
    for count, j in enumerate(dic_pat_adm[i]):
        if count != 0: continue
        for k in dic_admi_dgx[j[0]]:
            if k == '': continue
            dgx = icd_chapter_section(k)
            pat_dgx[i].append(dgx)
# drop patients that are 90 yr older
drop_list = []
for i in pat_dgx:
    if df_adm[df_adm['HADM_ID']==dic_pat_adm[i][-1][0]].age.values[0] > 90:
        drop_list.append(i)
        
# construct features and outcomes
X_total = np.zeros([len(pat_dgx), loc])
y_total = []
pat_idx = {}
n = 0
for i in pat_dgx:
    pat_idx[i] = n
    p = 0
    for dx in pat_dgx[i]:
        X_total[n, dx] += 1
        X_total[n, -1] = dic_pat_race[i]
        p += 1
    n += 1
    if i in df_cases.subject_id.values:
        y_total.append(1)
    else:
        y_total.append(0)
y_total = np.array(y_total)

# hyperparameters
range_concept = 121
n_fts_enhanced = 5
params = {
        'objective': 'binary',
        "max_depth": 24, 
        "learning_rate" : 0.005, 
        "num_leaves": 128,  
        "n_estimators": 1000,
        'n_jobs':-1,
        'subsample':0.90,
        'verbose': -1,
        'seed': 41
    }

In [3]:
lst_res_base = []
lst_res_new = []
dic_race_idx = {'white':0, 'black':1, 'asian':2, 'hispanic':3, 'other':4}
race_enhanced = 'black'
for seed in tqdm(range(100)):
    # split train/testing set
    X_train, X_test, y_train, y_test = train_test_split(X_total, y_total,
                                                        test_size=0.2, random_state=seed)
    # baseline model
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_model = lgb.train(params, lgb_train)
    y_pred = lgb_model.predict(X_test).reshape(-1, 1)
    fair_measure = Measure_disparity(X_train, X_test, y_train, y_test, y_pred, -1, dic_race_idx)
    dic_res = fair_measure.evaluate(2, race_enhanced)
    lst_res_base.append(dic_res)
    
    # find important features using Shapley values
    explainer = shap.TreeExplainer(model=lgb_model)
    X_test_sub = X_test[np.where(X_test[:,-1]==dic_race_idx[race_enhanced])[0], :]
    shap_values = explainer.shap_values(X_test_sub)
    lst_ft_imp_sort = np.argsort(np.mean(np.abs(shap_values[0]), axis=0))[::-1]
    # increase granularity on important features
    col_expand = [f for f in lst_ft_imp_sort if f <= range_concept][:n_fts_enhanced]
    col_expand = np.sort(col_expand)
    dic_col_expand_loc = {}
    loc = 150 # number of icd-9 sections
    for c in col_expand:
        res_reloc = relocate_icd9_section(c+1)
        dic_col_expand_loc[c] = (res_reloc[1], loc)
        loc += res_reloc[0]
    for i in dic_pat_adm:
        pat_dgx[i] = []
        count = 0
        for count, j in enumerate(dic_pat_adm[i]):
            if count != 0: continue
            for k in dic_admi_dgx[j[0]]:
                if k == '': continue
                dgx = icd_chapter_section(k)
                pat_dgx[i].append(dgx)
                if dgx in col_expand:
                    pat_dgx[i].append(int(k[:3])-dic_col_expand_loc[dgx][0]+dic_col_expand_loc[dgx][1])
    
    # construct features and outcomes
    X_total_new = np.zeros([len(pat_dgx), loc])
    y_total_new = []
    pat_idx = {}
    n = 0
    for i in pat_dgx:
        pat_idx[i] = n
        p = 0
        for dx in pat_dgx[i]:
            X_total_new[n, dx] += 1
            X_total_new[n, -1] = dic_pat_race[i]
            p += 1
        n += 1
        if i in df_cases.subject_id.values:
            y_total_new.append(1)
        else:
            y_total_new.append(0)
    y_total_new = np.array(y_total_new)

    # split data - identical split as training/validating the basemodel
    X_train, X_test, y_train, y_test = train_test_split(X_total_new, y_total_new,
                                                        test_size=0.2, random_state=seed)
    # model with increased granularity on important features
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_model = lgb.train(params, lgb_train)
    y_pred = lgb_model.predict(X_test).reshape(-1, 1)
    fair_measure = Measure_disparity(X_train, X_test, y_train, y_test, y_pred, -1, dic_race_idx)
    dic_res = fair_measure.evaluate(2, race_enhanced)
    lst_res_new.append(dic_res)

100%|███████████████████████████████████████| 100/100 [1:09:10<00:00, 41.50s/it]


In [19]:
dic_race_lst_res = {k:[[],[],[],[]] for k in [race_enhanced, 'Other', 'Disparity']}
for res in lst_res_base:
    for race in res:
        for i,v in enumerate(res[race]):
            dic_race_lst_res[race][i].append(v)
print("AUPR, AUROC before mitigation")
for race in [race_enhanced, 'Other']:
    auroc_mean, auroc_lower, auroc_upper = mean_confidence_interval(dic_race_lst_res[race][0])
    aupr_mean, aupr_lower, aupr_upper = mean_confidence_interval(dic_race_lst_res[race][1])
    print(f"{race}: {aupr_mean} ({aupr_lower}, {aupr_upper}), {auroc_mean} ({auroc_lower}, {auroc_upper})")
    
dic_race_lst_res = {k:[[],[],[],[]] for k in [race_enhanced, 'Other', 'Disparity']}
for res in lst_res_new:
    for race in res:
        for i,v in enumerate(res[race]):
            dic_race_lst_res[race][i].append(v)
print("\nAUPR, AUROC after mitigation")
for race in [race_enhanced, 'Other']:
    auroc_mean, auroc_lower, auroc_upper = mean_confidence_interval(dic_race_lst_res[race][0])
    aupr_mean, aupr_lower, aupr_upper = mean_confidence_interval(dic_race_lst_res[race][1])
    print(f"{race}: {aupr_mean} ({aupr_lower}, {aupr_upper}), {auroc_mean} ({auroc_lower}, {auroc_upper})")

AUPR, AUROC before mitigation
black: 0.137 (0.13, 0.143), 0.81 (0.804, 0.816)
Other: 0.17 (0.167, 0.172), 0.821 (0.82, 0.822)

AUPR, AUROC after mitigation
black: 0.144 (0.136, 0.151), 0.816 (0.81, 0.821)
Other: 0.171 (0.168, 0.173), 0.828 (0.827, 0.83)
